## Buinitski Stanislav 
## HW HEO_08 Task 2


The instance describes a hybrid flow shop

- every job must be processed sequentially through 5 stages,
- each stage contains a number of identical parallel machines,
- processing times differ per job and per stage.

The instance used  is:

- **10 jobs**
- **5 stages**
- **Identical machines per stage**: 3, 3, 2, 3, 3

**Processing times for the jobs at each stage are:**

- **Stage 1**: 5, 46, 90, 18, 18, 83, 96, 34, 33, 40
- **Stage 2**: 6, 20, 20, 68, 90, 49, 81, 97, 57, 28
- **Stage 3**: 55, 85, 87, 78, 60, 39, 50, 58, 94, 25
- **Stage 4**: 35, 72, 27, 48, 24, 15, 87, 23, 21, 95
- **Stage 5**: 77, 41, 52, 70, 48, 8, 63, 22, 67, 69

This instance fully matches the definition of a hybrid flow shop with identical machines and serves as the testbed for the discrete-event simulation and optimization experiments conducted in the following sections.

### Discrete Event Simulation Model

Each stage is a `Resource` with capacity = number of identical machines.

Each job is a SimPy process that:
- waits for a free machine at stage 1,
- processes for its stage-time,
- moves to stage 2, …, stage 5.

The permutation `perm` is your solution encoding:
- jobs are released into the system in that order,
- so when queues form, earlier jobs in `perm` tend to get served first.

**Makespan** = time when the last job finishes stage 5.

In [ ]:
import simpy, random, statistics

# ----- instance -----
n_jobs = 10
n_stages = 5
machines = [3, 3, 2, 3, 3]

pt = [
    [5,46,90,18,18,83,96,34,33,40],
    [6,20,20,68,90,49,81,97,57,28],
    [55,85,87,78,60,39,50,58,94,25],
    [35,72,27,48,24,15,87,23,21,95],
    [77,41,52,70,48,8,63,22,67,69],
]
# ----- DES evaluator of one permutation -----
def makespan_of(perm):
    env = simpy.Environment()
    stage_res = [simpy.Resource(env, capacity=machines[s]) for s in range(n_stages)]
    done = [0]*n_jobs

    def job(j):
        for s in range(n_stages):
            with stage_res[s].request() as req:
                yield req
                yield env.timeout(pt[s][j])
        done[j] = env.now

    for j in perm:                 # priority order
        env.process(job(j))

    env.run()
    return max(done)



In [ ]:
# ----- random baseline: 10 random solutions -----
random_makespans = []
for _ in range(10):
    perm = list(range(n_jobs))
    random.shuffle(perm)
    random_makespans.append(makespan_of(perm))

print("random makespans:", random_makespans)
print("average:", statistics.mean(random_makespans))

In [ ]:
# ----- Optimization using various heuristics -----

# 1. Shortest Processing Time (SPT) - sum of all stage times
def spt_heuristic():
    total_times = [sum(pt[s][j] for s in range(n_stages)) for j in range(n_jobs)]
    return sorted(range(n_jobs), key=lambda j: total_times[j])

# 2. Longest Processing Time (LPT)
def lpt_heuristic():
    total_times = [sum(pt[s][j] for s in range(n_stages)) for j in range(n_jobs)]
    return sorted(range(n_jobs), key=lambda j: total_times[j], reverse=True)

# 3. First stage priority (shortest first)
def first_stage_spt():
    return sorted(range(n_jobs), key=lambda j: pt[0][j])

# 4. Last stage priority (shortest first)
def last_stage_spt():
    return sorted(range(n_jobs), key=lambda j: pt[-1][j])

# 5. Bottleneck stage priority (stage with fewest machines)
def bottleneck_priority():
    bottleneck_stage = machines.index(min(machines))
    return sorted(range(n_jobs), key=lambda j: pt[bottleneck_stage][j])

# Test all heuristics
heuristics = {
    "SPT (total)": spt_heuristic(),
    "LPT (total)": lpt_heuristic(),
    "First stage SPT": first_stage_spt(),
    "Last stage SPT": last_stage_spt(),
    "Bottleneck priority": bottleneck_priority(),
}

print("Heuristic Results:")
print("-" * 50)
best_makespan = float('inf')
best_heuristic = None
best_perm = None

for name, perm in heuristics.items():
    ms = makespan_of(perm)
    print(f"{name:20s}: makespan = {ms:6.1f}")
    if ms < best_makespan:
        best_makespan = ms
        best_heuristic = name
        best_perm = perm

print("-" * 50)
print(f"Best heuristic: {best_heuristic}")
print(f"Best makespan: {best_makespan:.1f}")
print(f"Best permutation: {best_perm}")
print(f"Improvement over random avg: {(statistics.mean(random_makespans) - best_makespan):.1f}")


In [ ]:
# ----- Optuna optimization (vector encoding) -----
import optuna

# Parameters
OPTUNA_N_TRIALS = 50
OPTUNA_DIRECTION = 'minimize'

def optuna_objective(trial):
    x = [trial.suggest_float(f'x_{i}', 0.0, 1.0) for i in range(n_jobs)]
    perm = sorted(range(n_jobs), key=lambda j: x[j])
    return makespan_of(perm)

study = optuna.create_study(direction=OPTUNA_DIRECTION)
study.optimize(optuna_objective, n_trials=OPTUNA_N_TRIALS)
print('Optuna best makespan:', study.best_value)
best_x = [study.best_params[f'x_{i}'] for i in range(n_jobs)]
best_perm = sorted(range(n_jobs), key=lambda j: best_x[j])
print('Optuna best perm:', best_perm)

In [ ]:
# ----- scikit-optimize (gp_minimize) -----
from skopt import gp_minimize
from skopt.space import Real

# Parameters
SKOPT_N_CALLS = 50
SKOPT_RANDOM_STATE = 0
SKOPT_LOWER_BOUND = 0.0
SKOPT_UPPER_BOUND = 1.0

def skopt_objective(x):
    perm = sorted(range(n_jobs), key=lambda j: x[j])
    return makespan_of(perm)

space = [Real(SKOPT_LOWER_BOUND, SKOPT_UPPER_BOUND) for _ in range(n_jobs)]
res = gp_minimize(skopt_objective, space, n_calls=SKOPT_N_CALLS, random_state=SKOPT_RANDOM_STATE)
best_perm = sorted(range(n_jobs), key=lambda j: res.x[j])
print('Skopt best makespan:', res.fun)
print('Skopt best perm:', best_perm)

In [ ]:
# ----- DEAP Genetic Algorithm (permutation GA) -----
import random
from deap import base, creator, tools

# Parameters
GA_POP_SIZE = 20
GA_MUT_RATE = 0.2
GA_TOURN_SIZE = 3
GA_ELITISM = 2
GA_N_GEN = 600
GA_CROSSOVER_RATE = 0.9
GA_MUT_INDPB = 0.05

if not hasattr(creator, 'FitnessMin'):
    creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
if not hasattr(creator, 'Individual'):
    creator.create('Individual', list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register('individual', tools.initIterate, creator.Individual, lambda: random.sample(range(n_jobs), n_jobs))
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('evaluate', lambda ind: (makespan_of(ind),))
toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb=GA_MUT_INDPB)
toolbox.register('select', tools.selTournament, tournsize=GA_TOURN_SIZE)

pop = toolbox.population(n=GA_POP_SIZE)
for ind in pop:
    ind.fitness.values = toolbox.evaluate(ind)

for gen in range(GA_N_GEN):
    elites = tools.selBest(pop, GA_ELITISM)
    offspring = toolbox.select(pop, len(pop)-GA_ELITISM)
    offspring = list(map(toolbox.clone, offspring))
    for i in range(1, len(offspring), 2):
        if random.random() < GA_CROSSOVER_RATE:
            toolbox.mate(offspring[i-1], offspring[i])
            del offspring[i-1].fitness.values
            del offspring[i].fitness.values
    for mutant in offspring:
        if random.random() < GA_MUT_RATE:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    invalid = [ind for ind in offspring if not ind.fitness.valid]
    for ind in invalid:
        ind.fitness.values = toolbox.evaluate(ind)
    pop = elites + offspring

best = tools.selBest(pop, 1)[0]
print('DEAP best makespan:', best.fitness.values[0])
print('DEAP best perm:', best)

In [ ]:
# ----- Comparison of all optimization methods -----
import matplotlib.pyplot as plt

methods = ['Random', 'SPT', 'LPT', 'First SPT', 'Last SPT', 'Bottleneck', 'Optuna', 'Skopt', 'DEAP GA']
makespans = [
    statistics.mean(random_makespans),
    makespan_of(spt_heuristic()),
    makespan_of(lpt_heuristic()),
    makespan_of(first_stage_spt()),
    makespan_of(last_stage_spt()),
    makespan_of(bottleneck_priority()),
    study.best_value,
    res.fun,
    best.fitness.values[0]
]

plt.figure(figsize=(10, 5))
plt.bar(methods, makespans)
plt.ylabel('Makespan')
plt.xlabel('Model')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print(f"Best: {methods[makespans.index(min(makespans))]} = {min(makespans):.1f}")